# Load the Libraries

In [1]:
# importing libraries
import pandas as pd
import numpy as np

# Load the DataSet

In [2]:
# reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols,encoding='latin-1')

In [3]:
ratings

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [4]:
n_users = ratings.user_id.unique().shape[0]   #rows
n_items = ratings.movie_id.unique().shape[0]  #columns

In [5]:
print("The number of user:",n_users)
print("The number of n_items:",n_items)

The number of user: 943
The number of n_items: 1682


# Create pivot table for user and movie basedon ratings

In [6]:
datama=ratings.pivot_table(index='user_id',columns='movie_id',values='rating')

In [7]:
#Replace with nan with 0
data_matrix=datama.replace(np.nan,0)

In [8]:
data_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Find Cosine Similatity for user and Item

In [9]:
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')

# Using formula for user and item we are calcuating the score value

In [10]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [11]:
def predict(data_matrix, similarity, type='user'):
    if type == 'user':
        # Ensure proper conversion to NumPy array
        data_matrix = np.array(data_matrix)
        similarity = np.array(similarity)
        mean_user_rating = data_matrix.mean(axis=1)
        ratings_diff = data_matrix - mean_user_rating[:, None]  # Adjust indexing
        pred = mean_user_rating[:, None] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
        return pred
#[:, None] adds a new axis, converting a 1D array into a 2D column vector
    elif type == 'item':
        data_matrix = np.array(data_matrix)
        similarity = np.array(similarity)
        pred = data_matrix.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
        return pred


In [12]:
# prediction Table
##ValueError: Multi-dimensional indexing (predict cell)
#(e.g. `obj[:, None]`) is no longer supported. Convert to a numpy array before indexing instead.

In [13]:
# prediction Table
user_prediction = predict(data_matrix, user_similarity, type='user')
item_prediction = predict(data_matrix, item_similarity, type='item')

In [14]:
user_prediction

array([[ 2.06532606,  0.73430275,  0.62992381, ...,  0.39359041,
         0.39304874,  0.3927712 ],
       [ 1.76308836,  0.38404019,  0.19617889, ..., -0.08837789,
        -0.0869183 , -0.08671183],
       [ 1.79590398,  0.32904733,  0.15882885, ..., -0.13699223,
        -0.13496852, -0.13476488],
       ...,
       [ 1.59151513,  0.27526889,  0.10219534, ..., -0.16735162,
        -0.16657451, -0.16641377],
       [ 1.81036267,  0.40479877,  0.27545013, ..., -0.00907358,
        -0.00846587, -0.00804858],
       [ 1.8384313 ,  0.47964837,  0.38496292, ...,  0.14686675,
         0.14629808,  0.14641455]])

### As per User based filtering ,first have to find similarity between the input user and others

In [15]:
#1. Select input user 34

input_item=34


In [16]:
#2. Convert the user_sim table into DataFrame

item_sim_table=pd.DataFrame(item_similarity)

In [17]:
item_sim_table

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.000000,5.976178e-01,0.669755,0.545062,0.713286,0.883656,0.379021,0.518886,0.503712,0.726065,...,0.964613,1.0,1.000000,1.000000,0.964613,1.0,1.0,1.0,0.952817,0.952817
1,0.597618,1.110223e-16,0.726931,0.497429,0.681164,0.916437,0.616597,0.662998,0.744748,0.828918,...,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,0.921701,0.921701
2,0.669755,7.269308e-01,0.000000,0.675134,0.787043,0.893278,0.627079,0.799206,0.726331,0.841896,...,1.000000,1.0,1.000000,1.000000,0.967708,1.0,1.0,1.0,1.000000,0.903125
3,0.545062,4.974292e-01,0.675134,0.000000,0.665761,0.909692,0.510717,0.509764,0.580956,0.747439,...,1.000000,1.0,0.905978,0.905978,0.962391,1.0,1.0,1.0,0.943587,0.924782
4,0.713286,6.811638e-01,0.787043,0.665761,0.000000,0.962701,0.665231,0.740839,0.727552,0.944547,...,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,0.905789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,0.0,0.0,0.0,1.000000,1.000000
1678,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,0.0,0.0,0.0,1.000000,1.000000
1679,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,0.0,0.0,0.0,1.000000,1.000000
1680,0.952817,9.217006e-01,1.000000,0.943587,1.000000,1.000000,0.948502,0.917967,0.942640,1.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,0.000000,1.000000


In [18]:
#3. Find similarity user for 78 using cosine table

similar_input_item= item_sim_table[input_item].sort_values(ascending=True).head(5).index


In [19]:
similar_input_item

Index([34, 77, 246, 1030, 794], dtype='int64')

In [20]:
#4.Convert in to list

similar_item_input=list(similar_input_item)

In [21]:
#5. Using similar_item_input,can select movie id from ratings table

similar_item_userid_list=[]
for sim_item in similar_item_input:
    sim=list(ratings[ratings['movie_id']==sim_item]['user_id'])
    similar_item_userid_list.append(sim)

In [22]:
similar_item_userid_list

[[286, 276, 94, 184, 1, 551, 297],
 [92,
  222,
  224,
  194,
  59,
  291,
  262,
  64,
  83,
  128,
  106,
  135,
  334,
  301,
  276,
  184,
  188,
  201,
  110,
  234,
  256,
  286,
  320,
  372,
  345,
  244,
  381,
  60,
  375,
  42,
  330,
  56,
  43,
  115,
  270,
  417,
  280,
  405,
  183,
  343,
  94,
  474,
  347,
  313,
  416,
  271,
  269,
  497,
  506,
  109,
  504,
  7,
  246,
  429,
  363,
  346,
  76,
  95,
  72,
  399,
  311,
  566,
  496,
  374,
  618,
  450,
  505,
  293,
  318,
  593,
  524,
  457,
  328,
  332,
  659,
  650,
  178,
  1,
  655,
  299,
  453,
  643,
  308,
  268,
  49,
  653,
  454,
  664,
  766,
  528,
  532,
  437,
  741,
  487,
  215,
  551,
  764,
  577,
  586,
  478,
  371,
  533,
  378,
  521,
  774,
  500,
  721,
  389,
  711,
  881,
  545,
  843,
  823,
  633,
  627,
  870,
  815,
  916,
  893,
  455,
  130,
  561,
  749,
  145,
  647,
  303,
  393,
  394,
  442,
  495,
  267,
  65,
  682,
  690,
  922,
  896,
  186,
  156,
  452,
  554,
  9

In [23]:
#6. Convert all the list as single
import itertools
similar_item_userid_single_list=list(itertools.chain.from_iterable(similar_item_userid_list))

In [24]:
similar_item_userid_single_list

[286,
 276,
 94,
 184,
 1,
 551,
 297,
 92,
 222,
 224,
 194,
 59,
 291,
 262,
 64,
 83,
 128,
 106,
 135,
 334,
 301,
 276,
 184,
 188,
 201,
 110,
 234,
 256,
 286,
 320,
 372,
 345,
 244,
 381,
 60,
 375,
 42,
 330,
 56,
 43,
 115,
 270,
 417,
 280,
 405,
 183,
 343,
 94,
 474,
 347,
 313,
 416,
 271,
 269,
 497,
 506,
 109,
 504,
 7,
 246,
 429,
 363,
 346,
 76,
 95,
 72,
 399,
 311,
 566,
 496,
 374,
 618,
 450,
 505,
 293,
 318,
 593,
 524,
 457,
 328,
 332,
 659,
 650,
 178,
 1,
 655,
 299,
 453,
 643,
 308,
 268,
 49,
 653,
 454,
 664,
 766,
 528,
 532,
 437,
 741,
 487,
 215,
 551,
 764,
 577,
 586,
 478,
 371,
 533,
 378,
 521,
 774,
 500,
 721,
 389,
 711,
 881,
 545,
 843,
 823,
 633,
 627,
 870,
 815,
 916,
 893,
 455,
 130,
 561,
 749,
 145,
 647,
 303,
 393,
 394,
 442,
 495,
 267,
 65,
 682,
 690,
 922,
 896,
 186,
 156,
 452,
 554,
 932,
 243,
 758,
 864,
 693,
 847,
 498,
 796,
 897,
 719,
 889,
 50,
 269,
 99,
 276,
 79,
 150,
 63,
 89,
 323,
 32,
 58,
 367,
 268,
 1

In [25]:
#7. Unique movieid from the list

Unique_userid_similar_item=set(similar_item_userid_single_list)

In [26]:
len(Unique_userid_similar_item)

274

In [27]:
#8. Input user watched movie_list

input_item_watched_userid=list(ratings[ratings['movie_id']==input_item]['user_id'].values)

In [28]:
input_item_watched_userid

[286, 276, 94, 184, 1, 551, 297]

In [29]:
#9. Create a list which should have recom movieid to the input user

recom=[]
for per_id in Unique_userid_similar_item:
    if(per_id in input_item_watched_userid):
        pass
    else:
        recom.append(per_id)

In [30]:
len(recom)

267

In [31]:
sorted(recom)

[6,
 7,
 13,
 18,
 26,
 27,
 32,
 38,
 42,
 43,
 49,
 50,
 56,
 58,
 59,
 60,
 63,
 64,
 65,
 69,
 70,
 72,
 73,
 76,
 77,
 79,
 83,
 85,
 89,
 92,
 95,
 99,
 102,
 104,
 106,
 109,
 110,
 113,
 115,
 116,
 128,
 130,
 135,
 139,
 145,
 150,
 152,
 156,
 174,
 176,
 178,
 183,
 186,
 188,
 189,
 193,
 194,
 195,
 201,
 212,
 214,
 215,
 221,
 222,
 224,
 232,
 234,
 243,
 244,
 246,
 256,
 262,
 267,
 268,
 269,
 270,
 271,
 279,
 280,
 287,
 291,
 293,
 294,
 295,
 296,
 299,
 301,
 303,
 305,
 308,
 311,
 313,
 314,
 318,
 320,
 323,
 327,
 328,
 330,
 332,
 334,
 339,
 342,
 343,
 344,
 345,
 346,
 347,
 354,
 359,
 361,
 363,
 367,
 371,
 372,
 374,
 375,
 376,
 378,
 381,
 385,
 387,
 389,
 392,
 393,
 394,
 399,
 405,
 416,
 417,
 429,
 432,
 433,
 435,
 436,
 437,
 439,
 442,
 445,
 450,
 452,
 453,
 454,
 455,
 457,
 463,
 467,
 468,
 470,
 473,
 474,
 477,
 478,
 486,
 487,
 490,
 495,
 496,
 497,
 498,
 500,
 503,
 504,
 505,
 506,
 521,
 523,
 524,
 528,
 532,
 533,
 536,
 5

In [32]:
# Cross Checking 
sorted(Unique_userid_similar_item)

[1,
 6,
 7,
 13,
 18,
 26,
 27,
 32,
 38,
 42,
 43,
 49,
 50,
 56,
 58,
 59,
 60,
 63,
 64,
 65,
 69,
 70,
 72,
 73,
 76,
 77,
 79,
 83,
 85,
 89,
 92,
 94,
 95,
 99,
 102,
 104,
 106,
 109,
 110,
 113,
 115,
 116,
 128,
 130,
 135,
 139,
 145,
 150,
 152,
 156,
 174,
 176,
 178,
 183,
 184,
 186,
 188,
 189,
 193,
 194,
 195,
 201,
 212,
 214,
 215,
 221,
 222,
 224,
 232,
 234,
 243,
 244,
 246,
 256,
 262,
 267,
 268,
 269,
 270,
 271,
 276,
 279,
 280,
 286,
 287,
 291,
 293,
 294,
 295,
 296,
 297,
 299,
 301,
 303,
 305,
 308,
 311,
 313,
 314,
 318,
 320,
 323,
 327,
 328,
 330,
 332,
 334,
 339,
 342,
 343,
 344,
 345,
 346,
 347,
 354,
 359,
 361,
 363,
 367,
 371,
 372,
 374,
 375,
 376,
 378,
 381,
 385,
 387,
 389,
 392,
 393,
 394,
 399,
 405,
 416,
 417,
 429,
 432,
 433,
 435,
 436,
 437,
 439,
 442,
 445,
 450,
 452,
 453,
 454,
 455,
 457,
 463,
 467,
 468,
 470,
 473,
 474,
 477,
 478,
 486,
 487,
 490,
 495,
 496,
 497,
 498,
 500,
 503,
 504,
 505,
 506,
 521,
 523,

In [33]:
# Cross Checking
sorted(input_item_watched_userid)

[1, 94, 184, 276, 286, 297, 551]

In [34]:
# Checking the common movie list
list(set(Unique_userid_similar_item)&set(input_item_watched_userid))

[1, 551, 297, 276, 184, 94, 286]

In [35]:
item_pred=pd.DataFrame(item_prediction)

In [36]:
item_pred

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.446278,0.475473,0.505938,0.443633,0.512667,0.547939,0.446243,0.463059,0.474916,0.515821,...,0.580579,0.576202,0.582478,0.582478,0.575717,0.588155,0.588155,0.588155,0.573107,0.566696
1,0.108544,0.132957,0.125589,0.124932,0.131178,0.129005,0.110883,0.122223,0.109599,0.121525,...,0.135490,0.136546,0.134829,0.134829,0.134108,0.134458,0.134458,0.134458,0.136576,0.137111
2,0.085685,0.091690,0.087643,0.089966,0.089658,0.089985,0.083492,0.089725,0.085188,0.088331,...,0.089770,0.090506,0.086261,0.086261,0.089201,0.084659,0.084659,0.084659,0.089768,0.090845
3,0.053693,0.059604,0.058114,0.058364,0.059356,0.061472,0.053374,0.058615,0.055905,0.060601,...,0.061349,0.061686,0.061195,0.061195,0.060693,0.057937,0.057937,0.057937,0.061673,0.062281
4,0.224739,0.229171,0.263280,0.226387,0.259973,0.296529,0.232710,0.237109,0.258581,0.275076,...,0.297628,0.295990,0.299922,0.299922,0.298188,0.302051,0.302051,0.302051,0.293373,0.294309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.092574,0.113870,0.110211,0.112040,0.112768,0.123140,0.098578,0.110839,0.098858,0.118579,...,0.123829,0.124430,0.120776,0.120776,0.121360,0.125056,0.125056,0.125056,0.123470,0.124327
939,0.164358,0.184894,0.196502,0.164884,0.195860,0.209652,0.162840,0.165606,0.171761,0.194536,...,0.217536,0.215515,0.219136,0.219136,0.216173,0.218583,0.218583,0.218583,0.216582,0.216819
940,0.032300,0.045024,0.042924,0.043223,0.047493,0.051077,0.032761,0.042646,0.039399,0.047421,...,0.052762,0.053042,0.052692,0.052692,0.051514,0.053028,0.053028,0.053028,0.051910,0.052280
941,0.157779,0.174095,0.189000,0.163514,0.186140,0.194151,0.164910,0.156970,0.167038,0.181295,...,0.197537,0.194479,0.198479,0.198479,0.197969,0.199793,0.199793,0.199793,0.197394,0.200031


In [37]:
#s[187].values >=0.4  for user based

In [38]:
item_pred[input_item]     # for item based rec to user 

0      0.572414
1      0.139686
2      0.092378
3      0.062496
4      0.286815
         ...   
938    0.122120
939    0.217996
940    0.053046
941    0.198270
942    0.332126
Name: 34, Length: 943, dtype: float64

In [43]:
print(input_item_pred.shape)  # Check rows and columns
print(input_item_pred.columns)  # Check column names

(1, 943)
RangeIndex(start=0, stop=943, step=1)


In [44]:
for re in recom:
    if re in input_item_pred.columns:
        value = input_item_pred[re].values
        if value >= 1:
            highest_Rated.append(re)
    else:
        print(f"Column {re} not found in input_item_pred.")


Column 943 not found in input_item_pred.


In [45]:
highest_Rated = []
input_item_pre = pd.DataFrame(item_pred[input_item])
input_item_pred = input_item_pre.T

# Ensure recom only contains valid columns
valid_recom = [r for r in recom if r in input_item_pred.columns]

for re in valid_recom:
    value = input_item_pred[re].values
    if value >= 1:  # Check if the condition is satisfied
        highest_Rated.append(re)

print("Highest rated items:", highest_Rated)

Highest rated items: [654]


In [ ]:
#ValueError: 943 is not in range   # From recomd list select hightest rated film which would like by the user. Based on User prediction

#highest_Rated=[]
#input_item_pre=pd.DataFrame(item_pred[input_item])
#input_item_pred=input_item_pre.T
#for re in recom:
 #   value=input_item_pred[re].values
   # if(value>=1):
    #    highest_Rated.append(re)


In [46]:
len(highest_Rated)

1

In [47]:
# Checking the common movie list  should be empty
list(set(recom)&set(input_item_watched_userid))

[]

In [48]:
def itembased(input_item,item_similarity,item_predictions,similar_item_count,thres):
    #Convert the user_sim table into DataFrame
    item_sim_table=pd.DataFrame(item_similarity)
    #Find similarity user for 78 using cosine table
    similar_input_item= item_sim_table[input_item].sort_values(ascending=True).head(similar_item_count).index
    #Convert in to list
    similar_item_input=list(similar_input_item) 
    #Using similar_user_input,can select movie id from ratings table
    similar_item_userid_list=[]
    for sim_item in similar_item_input:
        sim=list(ratings[ratings['movie_id']==sim_item]['user_id'])
        similar_item_userid_list.append(sim)
    #Converting as a whole list
    import itertools
    similar_item_userid_single_list=list(itertools.chain.from_iterable(similar_item_userid_list))
    #Unique movieid from the list
    Unique_userid_similar_item=set(similar_item_userid_single_list)
    #Input user watched movie_list
    input_item_watched_userid=list(ratings[ratings['movie_id']==input_item]['user_id'].values)
    #Create a list which should have recom movieid to the input user
    recom=[]
    for per_id in Unique_userid_similar_item:
        if(per_id in input_item_watched_userid):
            pass
        else:
            recom.append(per_id)
    #From recommendation list selecting only hightest rated(predicted) value
    highest_Rated=[]
    user_pred=pd.DataFrame(item_prediction)
    input_item_pre=pd.DataFrame(user_pred[input_item])
    input_item_pred=input_item_pre.T
    for re in recom:
        value=input_item_pred[re].values
        if(value>=thres):
            highest_Rated.append(re)
    highest_Rated
    return highest_Rated
    

In [49]:
#def userbased(input_user,user_similarity,user_predictions,similar_user_count,similar_user_movieid_count,thres):
Recommended_user=itembased(5,item_similarity,item_pred,5,0.8)

In [50]:
len(Recommended_user)

1

In [51]:
Recommended_user

[449]